In [ ]:
! pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 90.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

import tensorflow_hub as hub

import pandas as pd

import tensorflow_text as text

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf

import numpy as np

In [ ]:
df = pd.read_excel('/content/ff.xlsx')
df.head()

,Column1,Content.Text,target
0,0,- عندي تذاكر دخول حديقة الحيوان | ✨\n- يوم الج...,False
1,1,- عندي تذاكر دخول حديقة الحيوان | ✨\n- يوم الج...,False
2,2,- عندي تذاكر دخول حديقة الحيوان | ✨\n- يوم الج...,False
3,3,""" إن الله وملائكته يصلون على النبي يا أيها الذ...",False
4,4,#Trio_Arabic_Night\n\nعندي ٣ تذاكر قولد 2 ب٦٠٠,False


In [ ]:
#Cleaning the text data
import pandas as pd
import re
import string
from bs4 import BeautifulSoup
import nltk
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import spacy

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Load spacy
nlp = spacy.load('en_core_web_sm')

def clean_string(text, stem="None"):

    final_string = ""

    # Make lower
    text = text.lower()

    # Remove line breaks
    text = re.sub(r'\n', '', text)

    # Remove puncuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("arabic")
    useless_words = useless_words + ['hi', 'im']

    text_filtered = [word for word in text if not word in useless_words]

    # Remove numbers
    text_filtered = [re.sub(r'\w*\d\w*', '', w) for w in text_filtered]

    # Stem or Lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer()
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_string = ' '.join(text_stemmed)

    return final_string

In [ ]:
clean_string('متوفره جميع خدمات تويتر ..\n⚠️دعم زيادة متابعي')

'متوفره خدمات تويتر ⚠️دعم زيادة متابعي'

In [ ]:
df.isnull().sum()
df.dropna(inplace = True)

In [ ]:

df['Content.Text'] = df['Content.Text'].values.astype(str)


In [ ]:
df['Content.Text'] = df['Content.Text'].apply(lambda x: clean_string(x))

In [ ]:
df.head()

,Column1,Content.Text,target
0,0,عندي تذاكر دخول حديقة الحيوان ✨ يوم الجمعة ✨ ...,False
1,1,عندي تذاكر دخول حديقة الحيوان ✨ يوم الجمعة ✨ ...,False
2,2,عندي تذاكر دخول حديقة الحيوان ✨ يوم الجمعة ✨ ...,False
3,3,الله وملائكته يصلون النبي ءآمنوا صلوا وسلموا ت...,False
4,4,trioarabicnightعندي تذاكر قولد,False


In [ ]:
df['target'].value_counts()

False    6327
True     5764
Name: target, dtype: int64

In [ ]:
print(str(round(6328/12061,2))+'%')

0.52%


In [ ]:
df['target'].unique()[1]

True

In [ ]:
# creating two dataframe false and true

df_true = df[df['target']==df['target'].unique()[1]]

df_false = df[df['target']==df['target'].unique()[0]]

print("true Dataset Shape:", df_true.shape)

print("false Dataset Shape:", df_false.shape)

true Dataset Shape: (5764, 3)
false Dataset Shape: (6327, 3)


In [ ]:
df_false_downsampled = df_false.sample(df_true.shape[0])
df_false_downsampled.shape

(5764, 3)

In [ ]:
df_balanced = pd.concat([df_true , df_false_downsampled])
df_balanced['target'].value_counts()

True     5764
False    5764
Name: target, dtype: int64

In [ ]:
# Encoding the target variable
df_balanced['ad'] = df_balanced['target'].apply(lambda x:1 if x==df['target'].unique()[1] else 0)
df_balanced.head()

,Column1,Content.Text,target,ad
6328,0,تجتمع صفقه كرستيانورونالدوالاهداف الفنية والاد...,True,1
6329,3,rt وصلت الان طائره كرستيانورونالدو العاصمه ال...,True,1
6330,4,فيديو إيجاز لأعمال الخلية الإنسانية للمقاومة ...,True,1
6331,5,rt جماهير النصر الراعي الرسمي ينساكم العموديل...,True,1
6332,6,rt جماهير النصر الراعي الرسمي ينساكم العموديل...,True,1


In [ ]:
df_balanced['target'].unique()[0]

True

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(df_balanced['Content.Text'], df_balanced['target'])

In [ ]:
bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [ ]:
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string, name = 'Inputs')
preprocessed_text = bert_preprocessor(text_input)
embeed = bert_encoder(preprocessed_text)
dropout = tf.keras.layers.Dropout(0.1, name = 'Dropout')(embeed['pooled_output'])
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'Dense')(dropout)

In [ ]:
model = tf.keras.Model(inputs = [text_input], outputs = [outputs])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Inputs (InputLayer)            [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['Inputs[0][0]']                 
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [ ]:
Metrics = [tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
           tf.keras.metrics.Precision(name = 'precision'),
           tf.keras.metrics.Recall(name = 'recall')
           ]

In [ ]:
model.compile(optimizer ='adam',
               loss = 'binary_crossentropy',
               metrics = Metrics)

In [ ]:
x_train = np.array([np.array(val) for val in X_train])
history = model.fit(x_train, y_train, epochs = 25)


Epoch 1/25
271/271 [==============================] - 106s 344ms/step - loss: 0.6469 - accuracy: 0.6432 - precision: 0.6593 - recall: 0.6022
Epoch 2/25
211/271 [======================>.......] - ETA: 20s - loss: 0.6031 - accuracy: 0.7038 - precision: 0.7128 - recall: 0.7003

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

model = Pipeline([('tfidf', TfidfVectorizer()), ('clf', MultinomialNB()),])

model = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC()),])

In [ ]:
model.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
predictions = model.predict(X_test)

91/91 [==============================] - 34s 362ms/step


In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics

conf_matrix = metrics.confusion_matrix(y_true=y_test, y_pred=predictions)


fig, ax = plt.subplots(figsize=(6, 6))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')

plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

ValueError: ignored

In [ ]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

       False       0.96      0.96      0.96      1582
        True       0.96      0.96      0.96      1582

    accuracy                           0.96      3164
   macro avg       0.96      0.96      0.96      3164
weighted avg       0.96      0.96      0.96      3164



In [ ]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.9611251580278128


In [ ]:
import joblib

In [ ]:
joblib.dump(model,'pipe.joblib')

['pipe.joblib']